In [33]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [34]:
# Device configuration: use GPU if available, otherwise fallback to CPU
# Konfigurasi perangkat: gunakan GPU jika tersedia, jika tidak gunakan CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)  # Print device being used (for debugging) / Cetak perangkat yang digunakan (untuk debugging)

cuda:0


In [35]:
# Hyperparameters: set training parameters
# Hyperparameter: atur parameter pelatihan
num_epochs = 100  # Number of complete passes through the dataset / Jumlah siklus lengkap melalui dataset
num_classes = 10  # Number of output classes (digits 0-9) / Jumlah kelas keluaran (digit 0-9)
batch_size = 256  # Number of samples processed before updating model parameters / Jumlah sampel yang diproses sebelum memperbarui parameter model
learning_rate = 0.001  # Step size used by the optimizer to adjust the model's weights / Ukuran langkah yang digunakan oleh optimizer untuk menyesuaikan bobot model

In [36]:
# Data augmentation with transforms: apply transformations to the training data to increase variety
# Augmentasi data dengan transformasi: terapkan transformasi pada data pelatihan untuk meningkatkan variasi
transform = transforms.Compose([
    transforms.RandomRotation(10),  # Randomly rotate the image by up to 10 degrees / Putar gambar secara acak hingga 10 derajat
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Randomly translate the image by up to 10% in both directions / Pindahkan gambar secara acak hingga 10% di kedua arah
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor / Konversi gambar menjadi tensor PyTorch
    transforms.Normalize((0.1307,), (0.3081,))  # Normalize using the mean and std for MNIST dataset / Normalisasi menggunakan rata-rata dan deviasi standar untuk dataset MNIST
])

In [37]:
# Download and load the MNIST dataset (train and test sets)
# Unduh dan muat dataset MNIST (set pelatihan dan pengujian)
train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,  # True = load the training set / True = muat set pelatihan
                                           transform=transform,  # Apply data augmentations / Terapkan augmentasi data
                                           download=True)  # Download the dataset if not available locally / Unduh dataset jika tidak tersedia secara lokal

test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train=False,  # False = load the test set / False = muat set pengujian
                                          transform=transforms.Compose([
                                              transforms.ToTensor(),  # Convert to PyTorch tensor (no augmentation for test set) / Konversi ke tensor PyTorch (tidak ada augmentasi untuk set pengujian)
                                              transforms.Normalize((0.1307,), (0.3081,))  # Normalize using the same values / Normalisasi menggunakan nilai yang sama
                                          ]))

# Data loader: load datasets in batches for training and testing
# Pemuat data: muat dataset dalam batch untuk pelatihan dan pengujian
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,  # Batch size for training / Ukuran batch untuk pelatihan
                                           shuffle=True)  # Shuffle training data to ensure random batches / Acak data pelatihan untuk memastikan batch yang acak

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,  # Batch size for testing / Ukuran batch untuk pengujian
                                          shuffle=False)  # No need to shuffle test data / Tidak perlu mengacak data pengujian

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:16<00:00, 603305.99it/s] 


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 113629.13it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:07<00:00, 217323.55it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 719512.36it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [38]:
# Improved Convolutional Neural Network (CNN) architecture for image classification
# Arsitektur Jaringan Saraf Konvolusional (CNN) yang ditingkatkan untuk klasifikasi gambar
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        # First convolutional block:
        # Input: 28x28x1 (grayscale), Output: 14x14x32
        # Blok konvolusi pertama:
        # Masukan: 28x28x1 (grayscale), Keluaran: 14x14x32
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),  # Convolution: 32 filters of size 5x5, padding 2 / Konvolusi: 32 filter berukuran 5x5, padding 2
            nn.BatchNorm2d(32),  # Batch normalization: normalize activations to improve convergence / Normalisasi batch: menormalkan aktivasi untuk meningkatkan konvergensi
            nn.ReLU(),  # Activation function: introduces non-linearity / Fungsi aktivasi: memperkenalkan non-linearitas
            nn.MaxPool2d(kernel_size=2, stride=2))  # Max pooling: reduce feature map size by 2x (downsampling) / Max pooling: mengurangi ukuran peta fitur menjadi 2x (downsampling)

        # Second convolutional block:
        # Input: 14x14x32, Output: 7x7x64
        # Blok konvolusi kedua:
        # Masukan: 14x14x32, Keluaran: 7x7x64
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),  # Convolution: 64 filters of size 5x5 / Konvolusi: 64 filter berukuran 5x5
            nn.BatchNorm2d(64),  # Batch normalization / Normalisasi batch
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))  # Max pooling

        # Third convolutional block (added for improved accuracy):
        # Input: 7x7x64, Output: 3x3x128
        # Blok konvolusi ketiga (ditambahkan untuk meningkatkan akurasi):
        # Masukan: 7x7x64, Keluaran: 3x3x128
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),  # Convolution: 128 filters of size 3x3 / Konvolusi: 128 filter berukuran 3x3
            nn.BatchNorm2d(128),  # Batch normalization / Normalisasi batch
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))  # Max pooling

        # Dropout layer: randomly set some weights to zero during training to prevent overfitting
        # Lapisan dropout: secara acak mengatur beberapa bobot menjadi nol selama pelatihan untuk mencegah overfitting
        self.dropout = nn.Dropout(0.5)

        # Fully connected layers (classifier):
        # Lapisan sepenuhnya terhubung (klasifier):
        self.fc1 = nn.Linear(3*3*128, 512)  # Input is flattened from 3x3x128, output is 512 / Masukan diratakan dari 3x3x128, keluaran adalah 512
        self.fc2 = nn.Linear(512, num_classes)  # Final output layer, 10 output classes / Lapisan keluaran akhir, 10 kelas keluaran

    def forward(self, x):
        out = self.layer1(x)  # Pass input through layer 1 / Lewatkan masukan melalui lapisan 1
        out = self.layer2(out)  # Pass through layer 2 / Lewatkan melalui lapisan 2
        out = self.layer3(out)  # Pass through additional layer 3 / Lewatkan melalui lapisan tambahan 3
        out = out.reshape(out.size(0), -1)  # Flatten the output from conv layers / Ratakan keluaran dari lapisan konvolusi
        out = self.dropout(out)  # Apply dropout / Terapkan dropout
        out = self.fc1(out)  # Pass through first fully connected layer / Lewatkan melalui lapisan sepenuhnya terhubung pertama
        out = self.fc2(out)  # Output from final fully connected layer / Keluaran dari lapisan sepenuhnya terhubung terakhir
        return out

In [39]:
# Create the model and move it to the GPU/CPU
# Buat model dan pindahkan ke GPU/CPU
model = ConvNet(num_classes).to(device)

# Loss function: Cross entropy loss, suitable for multi-class classification problems
# Fungsi kerugian: Kerugian entropi silang, cocok untuk masalah klasifikasi multi-kelas
criterion = nn.CrossEntropyLoss()

# Optimizer: AdamW optimizer with weight decay to prevent overfitting
# Optimizer: Optimizer AdamW dengan pengurangan bobot untuk mencegah overfitting
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Learning rate scheduler: Reduce learning rate by a factor of 0.7 every 3 epochs
# Penjadwal laju pembelajaran: Kurangi laju pembelajaran dengan faktor 0,7 setiap 3 epoch
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.7)

In [40]:
# Train the model
# Latih model
total_step = len(train_loader)  # Total number of steps (batches) per epoch / Total jumlah langkah (batch) per epoch
for epoch in range(num_epochs):
    model.train()  # Set model to training mode / Setel model ke mode pelatihan
    
    cumulative_loss = 0.0  # Initialize cumulative loss for the epoch
    
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)  # Move images to GPU/CPU / Pindahkan gambar ke GPU/CPU
        labels = labels.to(device)  # Move labels to GPU/CPU / Pindahkan label ke GPU/CPU

        # Forward pass: compute model predictions
        # Proses maju: hitung prediksi model
        outputs = model(images)
        loss = criterion(outputs, labels)  # Compute loss between predictions and true labels / Hitung kerugian antara prediksi dan label yang benar

        # Backward pass: compute gradients
        # Proses mundur: hitung gradien
        optimizer.zero_grad()  # Clear old gradients / Hapus gradien lama
        loss.backward()  # Backpropagation: compute gradients / Backpropagation: hitung gradien
        optimizer.step()  # Update model parameters / Perbarui parameter model
        
        cumulative_loss += loss.item()  # Accumulate the loss

    # Calculate average loss for the epoch
    avg_loss = cumulative_loss / total_step

    # Log the epoch number and average loss
    print(f'Epoch [{epoch + 1}/{num_epochs}], Avg Loss: {avg_loss:.4f}')

    scheduler.step()  # Step the scheduler at the end of each epoch to adjust learning rate / Langkah penjadwal di akhir setiap epoch untuk menyesuaikan laju pembelajaran


Epoch [1/100], Avg Loss: 0.3541
Epoch [2/100], Avg Loss: 0.0975
Epoch [3/100], Avg Loss: 0.0768
Epoch [4/100], Avg Loss: 0.0586
Epoch [5/100], Avg Loss: 0.0557
Epoch [6/100], Avg Loss: 0.0481
Epoch [7/100], Avg Loss: 0.0468
Epoch [8/100], Avg Loss: 0.0409
Epoch [9/100], Avg Loss: 0.0411
Epoch [10/100], Avg Loss: 0.0341
Epoch [11/100], Avg Loss: 0.0343
Epoch [12/100], Avg Loss: 0.0327
Epoch [13/100], Avg Loss: 0.0300
Epoch [14/100], Avg Loss: 0.0293
Epoch [15/100], Avg Loss: 0.0281
Epoch [16/100], Avg Loss: 0.0265
Epoch [17/100], Avg Loss: 0.0244
Epoch [18/100], Avg Loss: 0.0257
Epoch [19/100], Avg Loss: 0.0241
Epoch [20/100], Avg Loss: 0.0227
Epoch [21/100], Avg Loss: 0.0226
Epoch [22/100], Avg Loss: 0.0212
Epoch [23/100], Avg Loss: 0.0214
Epoch [24/100], Avg Loss: 0.0206
Epoch [25/100], Avg Loss: 0.0212
Epoch [26/100], Avg Loss: 0.0206
Epoch [27/100], Avg Loss: 0.0192
Epoch [28/100], Avg Loss: 0.0191
Epoch [29/100], Avg Loss: 0.0195
Epoch [30/100], Avg Loss: 0.0186
Epoch [31/100], Avg

In [41]:
# Test the model
# Uji model
model.eval()  # Set model to evaluation mode (disable dropout, use running averages for batchnorm) / Setel model ke mode evaluasi (nonaktifkan dropout, gunakan rata-rata berjalan untuk batchnorm)
with torch.no_grad():  # No need to compute gradients for testing / Tidak perlu menghitung gradien untuk pengujian
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)  # Get model predictions / Dapatkan prediksi model
        _, predicted = torch.max(outputs.data, 1)  # Select class with the highest score / Pilih kelas dengan skor tertinggi
        total += labels.size(0)  # Increment total number of images / Tambahkan total jumlah gambar
        correct += (predicted == labels).sum().item()  # Count correct predictions / Hitung prediksi yang benar

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 
    # Print test accuracy / Cetak akurasi pengujian

Test Accuracy of the model on the 10000 test images: 99.58 %


In [42]:
# Save the model checkpoint
# Simpan model checkpoint
torch.save(model.state_dict(), 'model_improved.ckpt')  # Save the model's state (weights) / Simpan status model (bobot)